<a href="https://colab.research.google.com/github/hkhong72/big_data/blob/main/221021_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

배열 - 메모리상의 연속적인 공간을 만들어놓고 씀
      중간의 데이터를 조회삽입등의 경우에 문제가 생김

Linked-list > 노드를 구성해서 각 데이터마다 포인터를 연결해서 줌

맵 - 키값에따라서 밸류를 지정
     다른 키, 동일한주소 X 
     주소가 지정한 범위바깥으로 가면안됨
     속도가 빨라야함 
     -->> hash 함수가 나옴

자바의 최상위 함수 - 오브젝트함수

오브젝트생성시 오브젝트마다 아이디부여




common 패키지에

GroupKeyComparator 클래스생성


In [ ]:
package com.adacho.common;

import org.apache.hadoop.io.WritableComparable;
import org.apache.hadoop.io.WritableComparator;

public class GroupKeyComparator extends WritableComparator{
	protected GroupKeyComparator() {
		super(DateKey.class,true);
	}

	@Override
	public int compare(WritableComparable a, WritableComparable b) {
		// TODO Auto-generated method stub
		DateKey k1 = (DateKey)a;
		DateKey k2 = (DateKey)b;
		return k1.getYear().compareTo(k2.getYear()); //년도만비교
	}
	
}


매퍼클래스에 

DelayCountMapperDateKey 클래스 생성

In [ ]:
package com.adacho.mapper;

import java.io.IOException;

import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.LongWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Mapper;

import com.adacho.common.AirlinePerformanceParser;
import com.adacho.common.DateKey;
import com.adacho.counter.DelayCounter;

public class DelayCountMapperDateKey extends Mapper<LongWritable, Text, DateKey, IntWritable>{
	private final static IntWritable outputValue = new IntWritable(1);
	private DateKey outputKey = new DateKey();
	
	@Override
	protected void map(LongWritable key, Text value, Mapper<LongWritable, Text, DateKey, IntWritable>.Context context)
			throws IOException, InterruptedException {
		// TODO Auto-generated method stub
		AirlinePerformanceParser parser = new AirlinePerformanceParser(value);

		
		if (parser.isDepartureDelayAvailable()) {
			if (parser.getDepartureDelayTime() > 0) {// 도착딜레이시간이양수면 -> 지연상황
				outputKey.setYear("D," + parser.getYear());
				outputKey.setMonth(parser.getMonth());
				context.write(outputKey, outputValue);
			} else if (parser.getDepartureDelayTime() == 0) {// 예정시간과 같은 정시에도착
				context.getCounter(DelayCounter.scheduled_departure).increment(1);
			} else if (parser.getDepartureDelayTime() < 0) {// 정시보다도 빨리도착
				context.getCounter(DelayCounter.early_departure).increment(1);
			}
		} else {// 도착 지연시간이 존재하지 않는다면 (데이터가 누락됬다면?)
			context.getCounter(DelayCounter.not_available_departure).increment(1);
		}
		
		
		if (parser.isArriveDelayAvailable()) {
			if (parser.getArriveDelayTime() > 0) { // 도착딜레이시간이양수면 -> 지연상황
				outputKey.setYear("A," + parser.getYear());
				outputKey.setMonth(parser.getMonth());
				context.write(outputKey, outputValue);
			} else if (parser.getArriveDelayTime() == 0) { // 예정시간과 같은 정시에도착
				context.getCounter(DelayCounter.scheduled_arrival).increment(1);
			} else if (parser.getArriveDelayTime() < 0) {// 정시보다도 빨리도착
				context.getCounter(DelayCounter.early_arrival).increment(1);
			}
		} else {// 도착 지연시간이 존재하지 않는다면 (데이터가 누락됬다면?)
			context.getCounter(DelayCounter.not_available_arrival).increment(1);
		}
		
	}
}


리듀서 패키지에

DelayCountReducerDateKey 클래스생성


In [ ]:
package com.adacho.reducer;

import java.io.IOException;

import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Reducer;
import org.apache.hadoop.mapreduce.lib.output.MultipleOutputs;

import com.adacho.common.DateKey;

public class DelayCountReducerDateKey extends Reducer<DateKey, IntWritable, DateKey, IntWritable>{
	private IntWritable result = new IntWritable();
	private MultipleOutputs<DateKey, IntWritable> mos;
	private DateKey outputKey = new DateKey();

	@Override
	protected void setup(Reducer<DateKey, IntWritable, DateKey, IntWritable>.Context context)
			throws IOException, InterruptedException {
		// TODO Auto-generated method stub
		mos = new MultipleOutputs<DateKey, IntWritable>(context);
	}

	@Override
	protected void reduce(DateKey key, Iterable<IntWritable> values,
			Reducer<DateKey, IntWritable, DateKey, IntWritable>.Context context) throws IOException, InterruptedException {
		// TODO Auto-generated method stub
		String[] columns = key.toString().split(","); //키를 문자열로바꾸고콤마 기준으로짜름

		int sum = 0;
		Integer bMonth = key.getMonth();		

		if(columns[0].equals("D")) {
			for(IntWritable value : values) {
				if(bMonth != key.getMonth()) {
					result.set(sum);
					outputKey.setYear(key.getYear().substring(2)); //1998년일때 인덱스2부터 끝까지. 즉, 98년을 뜻함
					outputKey.setMonth(bMonth);
					mos.write("departure", outputKey, result);
					sum = 0;
				}
				sum += value.get();
				bMonth = key.getMonth();
			}
			if(key.getMonth() == bMonth) {
				result.set(sum);
				outputKey.setYear(key.getYear().substring(2)); //1998년일때 인덱스2부터 끝까지. 즉, 98년을 뜻함
				outputKey.setMonth(bMonth);
				mos.write("departure", outputKey, result);
				sum = 0;
			}
		}else {
			for(IntWritable value : values) {
				if(bMonth != key.getMonth()) {
					result.set(sum);
					outputKey.setYear(key.getYear().substring(2)); //1998년일때 인덱스2부터 끝까지. 즉, 98년을 뜻함
					outputKey.setMonth(bMonth);
					mos.write("arrival", outputKey, result);
					sum = 0;
				}
				sum += value.get();
				bMonth = key.getMonth();
			}
			if(key.getMonth() == bMonth) {
				result.set(sum);
				outputKey.setYear(key.getYear().substring(2)); //1998년일때 인덱스2부터 끝까지. 즉, 98년을 뜻함
				outputKey.setMonth(bMonth);
				mos.write("arrival", outputKey, result);
				sum = 0;
			}
		}

	}
	@Override
	protected void cleanup(Reducer<DateKey, IntWritable, DateKey, IntWritable>.Context context)
			throws IOException, InterruptedException {
		// TODO Auto-generated method stub
		mos.close(); //메모리릭발생방지를 위해 닫아줘야 함
	}
}


드라이버 패키지에 

DelayCountDateKey 클래스 생성


In [ ]:
package com.adacho.driver;

import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.conf.Configured;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.input.TextInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;
import org.apache.hadoop.mapreduce.lib.output.MultipleOutputs;
import org.apache.hadoop.mapreduce.lib.output.TextOutputFormat;
import org.apache.hadoop.util.GenericOptionsParser;
import org.apache.hadoop.util.Tool;
import org.apache.hadoop.util.ToolRunner;

import com.adacho.common.DateKey;
import com.adacho.common.DateKeyComparator;
import com.adacho.common.GroupKeyComparator;
import com.adacho.common.GroupKeyPartitioner;
import com.adacho.mapper.DelayCountMapperDateKey;
import com.adacho.reducer.DelayCountReducerDateKey;

public class DelayCountDateKey extends Configured implements Tool{
	public static void main(String[] args) throws Exception{
		int res = ToolRunner.run(new Configuration(), new DelayCountDateKey(), args);
		System.out.println("MapReduce Result : " + res);
	}
	
	@Override
	public int run(String[] args) throws Exception {
		// TODO Auto-generated method stub
		String[] otherArgs = new GenericOptionsParser(getConf(), args).getRemainingArgs();
		if(otherArgs.length !=2) {
			System.err.println("Usage: DelayCountDateKey <input> <output>");
			System.exit(1);
		}
		
		Job job = Job.getInstance(getConf(), "DelayCountDateKey");
		
		job.setJarByClass(DelayCountDateKey.class); //드라이브 클래스 저장
		job.setMapperClass(DelayCountMapperDateKey.class); //메퍼클래스저장
		job.setReducerClass(DelayCountReducerDateKey.class); //리듀스클래스저장
		
		job.setInputFormatClass(TextInputFormat.class); //두개중에 길이가 긴 것 임포트
		job.setOutputFormatClass(TextOutputFormat.class); //두개중에 길이가 긴 것 임포트
		
		job.setMapOutputKeyClass(DateKey.class);
		job.setMapOutputValueClass(IntWritable.class);
		
		FileInputFormat.addInputPath(job, new Path(otherArgs[0])); //여러개에 인풋데이타가 있으면 다 줄 수 있음
		FileOutputFormat.setOutputPath(job, new Path(otherArgs[1]));
		
		MultipleOutputs.addNamedOutput(job, "departure", TextOutputFormat.class, DateKey.class, IntWritable.class);
		MultipleOutputs.addNamedOutput(job, "arrival", TextOutputFormat.class, DateKey.class, IntWritable.class);
		
		job.setPartitionerClass(GroupKeyPartitioner.class);
		job.setGroupingComparatorClass(GroupKeyComparator.class);
		job.setSortComparatorClass(DateKeyComparator.class);
		
		job.waitForCompletion(true); //job이끝날때까지 기다리고있다가 동기방식으로
		
		return 0;
	}
}


maven clean , install

scp .\airPerformance-0.1.jar ubuntu:~/work/java

/work/java에서

yarn jar ./airPerformance-0.1.jar com.adacho.driver.DelayCountDateKey air-input delay-count-sort

hdfs dfs -ls delay-count-sort

hdfs dfs -cat delay-count-sort/arrival-r-00000

hdfs dfs -cat delay-count-sort/departure-r-00000
로 정렬확인


수업 끝

월요일 하둡에코시스템

화요일 딥러닝 예정
